# Pandas Basics — 75 Worked Examples (15 per section)

Sections:
1) **Pandas Basics**
2) **Introduction to Pandas**
3) **Series and DataFrame objects**
4) **Reading/Writing data from CSV/Excel/JSON**
5) **Data selection, filtering, indexing**

Each example:
- Markdown explanation (what the function is about)
- Code cell
- Optimization tips (markdown)


In [ ]:
import numpy as np
import pandas as pd
np.__version__, pd.__version__

## Section 1 — Pandas Basics (15 Examples)

### Create `Series` and `DataFrame`
Core labeled structures.

In [ ]:
s = pd.Series([10,20,30], index=list('abc'))
df = pd.DataFrame({'A':[1,2,3],'B':[4.5,5.5,6.5]})
print(s); print(df)

**Optimization options**
- Name columns/indexes; prefer numeric dtypes.

### Inspect: `info`, `dtypes`, `shape`
Columns, nulls, memory, dtypes.

In [ ]:
print(df.info()); print(df.dtypes); print(df.shape)

**Optimization options**
- Inspect early; fix dtypes.

### Peek: `head`, `tail`, `sample`
First/last N rows & sampling.

In [ ]:
print(df.head(2)); print(df.tail(1)); print(df.sample(1, random_state=0))

**Optimization options**
- Use small `head()`; `sample` with seed.

### Describe: `describe`, `value_counts`
Stats and frequencies.

In [ ]:
print(df.describe()); print(pd.Series(list('aabccc')).value_counts())

**Optimization options**
- Combine stats in one pass.

### Rename: `rename`, `set_axis`
Safe renaming without inplace.

In [ ]:
ren = df.rename(columns={'A':'alpha','B':'beta'}); print(ren); print(ren.set_axis(['p','q'], axis=1))

**Optimization options**
- Avoid `inplace=True`; chain ops.

### Cast: `astype`, `to_numeric`
Fix mixed types for math.

In [ ]:
mixed = pd.Series(['1','2','bad','3']); print(pd.to_numeric(mixed, errors='coerce')); print(df['A'].astype('int32').dtype)

**Optimization options**
- Cast once; sanitize with `to_numeric`.

### Vectorized ops & alignment
Operate column-wise; indexes align.

In [ ]:
df2 = pd.DataFrame({'A':[10,20,30],'C':[1,1,1]}); print((df[['A']]+df2[['A']]).assign(C=df2['C']))

**Optimization options**
- Leverage alignment over manual joins.

### Boolean masks
Filter or update conditionally.

In [ ]:
m = df['A']>1; print(df[m])

**Optimization options**
- Reuse masks; combine with `&`/`|`/`~`.

### Missing values
Detect/impute simple gaps.

In [ ]:
m = pd.Series([1.0, None, 3.0]); print(m.isna()); print(m.fillna(m.mean()))

**Optimization options**
- Impute with domain-aware rules.

### Sorting
Order by values or index.

In [ ]:
print(df.sort_values('B', ascending=False)); print(df.sort_index(ascending=False))

**Optimization options**
- Stable sort when order matters.

### Index control
`set_index`/`reset_index`.

In [ ]:
idx_df = pd.DataFrame({'k':[1,2,3],'v':[10,20,30]}).set_index('k'); print(idx_df); print(idx_df.reset_index())

**Optimization options**
- Plan index strategy early.

### Concat basics: `pd.concat`
Stack rows or columns.

In [ ]:
a = pd.DataFrame({'k':[1,2], 'v':['a','b']}); b = pd.DataFrame({'k':[3], 'v':['c']}); print(pd.concat([a,b], ignore_index=True))

**Optimization options**
- `ignore_index=True` when stacking.

### Quick merge: `pd.merge`
Join on keys (inner).

In [ ]:
L = pd.DataFrame({'k':[1,2,3],'L':['a','b','c']}); R = pd.DataFrame({'k':[2,3,4],'R':['u','v','w']}); print(pd.merge(L,R,on='k'))

**Optimization options**
- Match key dtypes before merge.

### GroupBy intro
Split-apply-combine.

In [ ]:
gdf = pd.DataFrame({'k':['A','A','B','B'], 'v':[1,2,3,4]}); print(gdf.groupby('k')['v'].agg(['mean','sum']))

**Optimization options**
- Use `agg` for multiple stats.

### Copy vs view & assignment
Avoid chained assignment.

In [ ]:
base = pd.DataFrame({'x':[1,2,3]}); view = base['x']; safe = base['x'].copy(); base.loc[base['x']>1,'x']=99; print(base, view.iloc[0], safe.iloc[0])

**Optimization options**
- Use `.loc` for writes; `.copy()` when in doubt.

## Section 2 — Introduction to Pandas (15 Examples)

### `assign` pipelines
Add columns in a chain.

In [ ]:
df = pd.DataFrame({'A':[1,2,3],'B':[4,5,6]}); print(df.assign(C=lambda d:d['A']+d['B']))

**Optimization options**
- Keep chains readable; compute once.

### `pipe`
Composable functions.

In [ ]:
def f(d): d=d.copy(); d['r']=d['A']/d['B']; return d
print(pd.DataFrame({'A':[1,2,3],'B':[4,5,6]}).pipe(f))

**Optimization options**
- Test functions separately; pass args via `pipe`.

### `query`
Readable filters.

In [ ]:
df = pd.DataFrame({'A':[1,2,3,4],'B':[10,20,10,30]}); print(df.query('A>=2 and B!=10'))

**Optimization options**
- Use backticks for awkward column names.

### `eval`
Expression evaluation.

In [ ]:
df = pd.DataFrame({'x':[1,2,3],'y':[4,5,6]}); df.eval('z=x+y', inplace=True); print(df)

**Optimization options**
- Benchmark; benefits depend on expr/data.

### Categorical
Memory & semantics.

In [ ]:
cat = pd.Series(['low','med','high','low']).astype('category'); print(cat.dtype, cat.cat.categories)

**Optimization options**
- Set category order for sorting/grouping.

### Datetime parse
`to_datetime`.

In [ ]:
ts = pd.to_datetime(['2025-01-01','2025-01-03']); print(ts, ts.dtype)

**Optimization options**
- Normalize timezones; store datetime64[ns].

### Resample
Aggregate to new freq.

In [ ]:
idx = pd.date_range('2025-01-01', periods=10, freq='D'); s = pd.Series(range(10), index=idx); print(s.resample('W').mean())

**Optimization options**
- Ensure DatetimeIndex; choose right agg.

### Rolling
Moving stats.

In [ ]:
s = pd.Series([1,2,3,4,5]); print(s.rolling(3).mean())

**Optimization options**
- Choose window & min_periods; mind edges.

### Pivot table
Aggregate matrix.

In [ ]:
d = pd.DataFrame({'k':['A','A','B'],'c':['x','y','x'],'v':[1,2,3]}); print(pd.pivot_table(d, index='k', columns='c', values='v', aggfunc='sum', fill_value=0))

**Optimization options**
- `fill_value` to avoid NaNs.

### Melt
Unpivot wide->long.

In [ ]:
w = pd.DataFrame({'id':[1,2], 'A':[10,20], 'B':[30,40]}); print(pd.melt(w, id_vars='id'))

**Optimization options**
- Keep id vars explicit.

### String ops
.str vectorization.

In [ ]:
text = pd.Series(['a_a','b-b']); print(text.str.upper().str.replace('[^A-Z]','', regex=True))

**Optimization options**
- Precompile regex for heavy use.

### `value_counts` & `nunique`
Frequencies & distincts.

In [ ]:
s = pd.Series(list('aabcc')); print(s.value_counts()); print(s.nunique())

**Optimization options**
- For grouped freqs: `groupby().value_counts()`.

### Memory & downcast
Estimate/optimize.

In [ ]:
df = pd.DataFrame({'i':[1,2,3], 'f':[1.0,2.0,3.0]}); print(df.memory_usage(deep=True)); print(pd.to_numeric(pd.Series([1,2,3]), downcast='integer').dtype)

**Optimization options**
- Downcast ints/floats where safe.

### Display options
`set_option`.

In [ ]:
pd.set_option('display.precision', 3); print(pd.DataFrame({'x':[1.23456]}))

**Optimization options**
- Scope display changes.

### Row-wise `apply`
Use sparingly.

In [ ]:
df = pd.DataFrame({'A':[1,2,3],'B':[4,5,6]}); df['sum']=df.apply(lambda r:r['A']+r['B'], axis=1); print(df)

**Optimization options**
- Prefer vectorization over `apply`.

## Section 3 — Series and DataFrame objects (15 Examples)

### Constructors & indices
Specify index/columns.

In [ ]:
s = pd.Series(np.arange(3), index=list('abc')); df = pd.DataFrame({'A':np.arange(3), 'B':list('xyz')}); print(s); print(df)

**Optimization options**
- Pass index/columns explicitly.

### Attributes & accessors
index/columns/values; .str/.dt/.cat.

In [ ]:
print(df.index, df.columns); text = pd.Series(['id=1','id=20']); print(text.str.extract(r'id=(\d+)'))

**Optimization options**
- Prefer vectorized accessors; precompile regex.

### Select columns/rows
Use .loc/.iloc.

In [ ]:
print(df['A']); print(df.loc[1])

**Optimization options**
- Avoid chained indexing.

### Insert/assign/drop
Predictable column ops.

In [ ]:
g = df.copy(); g.insert(1,'C',g['A']*10); g=g.assign(D=lambda d:d['A']+1); print(g.drop(columns=['D']))

**Optimization options**
- `insert` avoids reallocation; `assign` aids chaining.

### Dtypes & casting
Sanitize to numeric.

In [ ]:
mixed = pd.Series(['1','2','bad','3']); print(pd.to_numeric(mixed, errors='coerce'))

**Optimization options**
- Use `errors='coerce'`.

### Map/replace
Value mapping/substitution.

In [ ]:
s = pd.Series(['low','med','high']); print(s.map({'low':0,'med':1,'high':2})); print(s.replace({'low':'L'}))

**Optimization options**
- `map` for 1:1; `replace` for patterns.

### where/mask
Conditional replacement.

In [ ]:
ser = pd.Series([1,2,3,4]); print(ser.where(ser%2==0, -1)); print(ser.mask(ser%2==0, 0))

**Optimization options**
- Replace branches with `where`/`mask`.

### combine_first
Fill NAs from right.

In [ ]:
a = pd.Series([1, np.nan, 3], index=list('abc')); b = pd.Series([9,8,7], index=list('bcd')); print(a.combine_first(b))

**Optimization options**
- Preserve left; fill gaps safely.

### Reindex
Align to new index.

In [ ]:
base = pd.Series([1,2,3], index=list('abc')); print(base.reindex(list('acde'), fill_value=0))

**Optimization options**
- Fill carefully to avoid masking true NaNs.

### Duplicates & uniques
Detect/remove, list uniques.

In [ ]:
vals = pd.Series([1,1,2,3,3,3]); print(vals.duplicated()); print(vals.unique())

**Optimization options**
- Use `duplicated(keep='first')`; `nunique()`.

### clip/round/abs
Bound/round/abs.

In [ ]:
num = pd.Series([-3.2,-1.5,0.2,4.8]); print(num.clip(-2,2).round(1).abs())

**Optimization options**
- `clip` and round late.

### pivot & pivot_table
Reshape & aggregate.

In [ ]:
d = pd.DataFrame({'k':['A','A','B'],'c':['x','y','x'],'v':[1,2,3]}); print(d.pivot(index='k', columns='c', values='v')); print(pd.pivot_table(d, index='k', columns='c', values='v', aggfunc='sum'))

**Optimization options**
- `pivot_table` handles duplicates.

### rolling/expanding
Moving & cumulative stats.

In [ ]:
w = pd.Series([1,2,3,4,5]); print(w.rolling(3).mean()); print(w.expanding(2).sum())

**Optimization options**
- Choose window/min_periods wisely.

### String contains/extract
.str with regex.

In [ ]:
tokens = pd.Series(['id=123','id=999','noid']); print(tokens.str.contains(r'id=\d+')); print(tokens.str.extract(r'id=(\d+)'))

**Optimization options**
- Use `.str` efficiently; literal replace with `regex=False` when possible.

### MultiIndex & xs
Build multi-level index; cross-sections.

In [ ]:
mi = pd.MultiIndex.from_product([['A','B'],[1,2]], names=['grp','id']); M = pd.DataFrame({'v':[10,20,30,40]}, index=mi); print(M.xs('A', level='grp'))

**Optimization options**
- Name levels; `xs` for fast slices.

## Section 4 — Reading/Writing data from CSV/Excel/JSON (15 Examples)

### read_csv basics
Auto inference & header.

In [ ]:
csv='/mnt/data/pandas_io_demo/basic.csv'; pd.DataFrame({'A':[1,2],'B':[3.5,4.5]}).to_csv(csv, index=False); print(pd.read_csv(csv))

**Optimization options**
- Decide dtypes at read-time.

### usecols/dtype/na_values
Select & sanitize on read.

In [ ]:
p='/mnt/data/pandas_io_demo/typed.csv'; pd.DataFrame({'id':[1,2,3],'score':['9.1','NA','8.0']}).to_csv(p,index=False); print(pd.read_csv(p,usecols=['id','score'],dtype={'id':'int32','score':'float32'},na_values=['NA']))

**Optimization options**
- Use `usecols` to cut I/O.

### parse_dates
Datetime on ingest.

In [ ]:
p='/mnt/data/pandas_io_demo/dates.csv'; pd.DataFrame({'date':['2025-01-01','2025-01-03'],'y':[10,20]}).to_csv(p,index=False); print(pd.read_csv(p, parse_dates=['date']))

**Optimization options**
- Parse dates on ingest.

### chunksize
Stream large CSV.

In [ ]:
p='/mnt/data/pandas_io_demo/big.csv'; pd.DataFrame({'x':range(1000)}).to_csv(p,index=False); s=0
for c in pd.read_csv(p,chunksize=200): s+=c['x'].sum()
print(s)

**Optimization options**
- Stream with `chunksize`.

### to_csv compression
Write gzip.

In [ ]:
p='/mnt/data/pandas_io_demo/file.csv.gz'; pd.DataFrame({'A':[1,2,3]}).to_csv(p,index=False,compression='gzip'); print(p)

**Optimization options**
- Compress outputs to save space.

### read_json/to_json
Round-trip JSON.

In [ ]:
p='/mnt/data/pandas_io_demo/data.json'; df=pd.DataFrame({'id':[1,2],'name':['A','B']}); df.to_json(p, orient='records'); print(pd.read_json(p, orient='records'))

**Optimization options**
- Pick `orient` that matches downstream.

### read_excel/to_excel
Excel I/O (engine needed).

In [ ]:
p='/mnt/data/pandas_io_demo/demo.xlsx'; df=pd.DataFrame({'A':[1,2],'B':[3,4]}); print('Excel path:', p)

**Optimization options**
- Pin engines for reproducibility.

### json_normalize
Flatten nested JSON.

In [ ]:
nested=[{'id':1,'meta':{'os':'win','ver':11}},{'id':2,'meta':{'os':'linux','ver':6}}]; print(pd.json_normalize(nested))

**Optimization options**
- Flatten nested JSON early.

### read_html
Parse HTML tables.

In [ ]:
print('read_html demo (requires libs)')

**Optimization options**
- Cache/limit HTML parsing.

### clipboard I/O
Quick copy/paste.

In [ ]:
print('clipboard demo (GUI needed)')

**Optimization options**
- Prefer files over clipboard.

### Parquet I/O
Efficient columnar.

In [ ]:
print('parquet demo (engine needed)')

**Optimization options**
- Prefer Parquet for analytics.

### to_json compressed
JSON + gzip.

In [ ]:
p='/mnt/data/pandas_io_demo/data.json.gz'; pd.DataFrame({'x':[1,2,3]}).to_json(p, orient='records', compression='gzip'); print(p)

**Optimization options**
- Compress when shipping.

### custom delimiter
TSV/pipe files.

In [ ]:
p='/mnt/data/pandas_io_demo/data.tsv'; pd.DataFrame({'A':[1,2],'B':[3,4]}).to_csv(p, sep='\t', index=False); print(pd.read_csv(p, sep='\t'))

**Optimization options**
- Specify `sep`/`encoding` explicitly.

### bad lines
Handle malformed rows.

In [ ]:
p='/mnt/data/pandas_io_demo/bad.csv'; open(p,'w').write('A,B\n1,2\n3,4,5  # bad\n5,6\n'); print(pd.read_csv(p, on_bad_lines='skip'))

**Optimization options**
- Set policy for bad lines.

### explicit NA rules
Control NA tokens.

In [ ]:
p='/mnt/data/pandas_io_demo/na.csv'; pd.DataFrame({'A':['','NA','ok'],'B':[1,2,3]}).to_csv(p, index=False); print(pd.read_csv(p, keep_default_na=False, na_values=['NA']))

**Optimization options**
- Control NA parsing explicitly.

## Section 5 — Data selection, filtering, indexing (15 Examples)

### Label-based `.loc`
Select by labels.

In [ ]:
df = pd.DataFrame({'A':[1,2,3],'B':[4,5,6]}, index=['r1','r2','r3']); print(df.loc['r2','B']); print(df.loc['r1':'r3',['A']])

**Optimization options**
- Prefer `.loc` with explicit rows/cols.

### Position-based `.iloc`
Integer positions.

In [ ]:
print(df.iloc[1,0]); print(df.iloc[:,1])

**Optimization options**
- `.iloc` removes ambiguity.

### Fast scalars `.at`/`.iat`
Optimized element access.

In [ ]:
print(df.at['r3','A'], df.iat[0,1])

**Optimization options**
- Use `.at`/`.iat` for tight loops.

### Boolean filtering
Filter with masks.

In [ ]:
crit = df['A']>=2; print(df[crit])

**Optimization options**
- Reuse masks; combine operators.

### `query`
SQL-like filters.

In [ ]:
print(df.query('A>=2 and B<6'))

**Optimization options**
- Use backticks for odd names.

### `isin`
Set membership filters.

In [ ]:
print(df[df['A'].isin([1,3])])

**Optimization options**
- Categorical for repeated `isin`.

### `between`
Inclusive ranges.

In [ ]:
s = pd.Series([10,15,20,25]); print(s[s.between(15,22)])

**Optimization options**
- Vectorized and inclusive.

### `where`/`mask`
Conditional replacement.

In [ ]:
ser = pd.Series([1,2,3,4]); print(ser.where(ser%2==0, -1)); print(ser.mask(ser%2==0, 0))

**Optimization options**
- Preserve shape with `where`/`mask`.

### `reindex`
Align to new index.

In [ ]:
base = pd.Series([1,2,3], index=list('abc')); print(base.reindex(list('acde'), fill_value=0))

**Optimization options**
- Align via `reindex` carefully.

### MultiIndex & `xs`
Cross-sections on levels.

In [ ]:
mi = pd.MultiIndex.from_product([['A','B'],[1,2]], names=['grp','id']); M = pd.DataFrame({'v':[10,20,30,40]}, index=mi); print(M.xs('A', level='grp'))

**Optimization options**
- Name levels; use `xs`.

### Sort values/index
Ordering for joins/slicing.

In [ ]:
print(M.sort_values('v', ascending=False)); print(M.sort_index())

**Optimization options**
- Stable sorts for deterministic order.

### Top-k `nlargest`/`nsmallest`
Fast top/bottom k.

In [ ]:
print(M['v'].nlargest(2))

**Optimization options**
- Avoid full sorts.

### Duplicates
Find & drop duplicates.

In [ ]:
D = pd.DataFrame({'k':[1,1,2,3],'v':[9,9,8,7]}); print(D.duplicated()); print(D.drop_duplicates())

**Optimization options**
- De-duplicate early.

### `take` & `sample`
Positional and random picks.

In [ ]:
print(D.take([0,2])); print(D.sample(n=2, random_state=1))

**Optimization options**
- `take` is efficient; `sample` with seed.

### Index set/reset chain
Predictable slicing.

In [ ]:
chain = (D.set_index('k').sort_index().loc[1:2]); print(chain)

**Optimization options**
- Consistent indexing + sorting speeds slices.